# Global registration with RANSAC
We are going to use open3d (http://www.open3d.org/) to handle  pointclouds and generation of pointclouds


In [1]:
import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


We need to read in our pointclouds.

In [2]:
source = o3d.io.read_point_cloud("ICP/r1.pcd")
target = o3d.io.read_point_cloud("ICP/r2.pcd")

# Used for downsampling.
voxel_size = 0.05

# Show models side by side
draw_registrations(source, target)

### Finding features in pointclouds
When working on point clouds it can be benefitial work on a downsampled version of the point cloud.
you can use [```pointcloudname.voxel_down_sample()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html) where pointcloud is the name of your point cloud object.

We also need to estimate the normals of the pointcloud points using [```pointcloudname.estimate_normals()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html)

And finally find fpfh features or correspondance of the downsampled point clouds.
[```o3d.pipelines.registration.compute_fpfh_feature()```](http://www.open3d.org/docs/latest/python_api/open3d.pipelines.registration.compute_fpfh_feature.html)


In [4]:
####
# Downsample and find features here
####

# Code

# Load
source = o3d.io.read_point_cloud("ICP/r1.pcd")
target = o3d.io.read_point_cloud("ICP/r2.pcd")

# Used for downsampling.
voxel_size = 0.05

# Code
source_sample = source.voxel_down_sample(voxel_size)
target_sample = target.voxel_down_sample(voxel_size)

default_tree = o3d.geometry.KDTreeSearchParamHybrid(radius=10,max_nn=60)

source_sample.estimate_normals()
target_sample.estimate_normals()

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source_sample, default_tree)
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target_sample, default_tree)

print(len(source_sample.points))
print(len(target_sample.points))

4760
3440


### Ransac
We will now attempt to use ransac to do a global registration of the two poinclouds.

Using the function [```o3d.pipelines.registration.registration_ransac_based_on_feature_matching```](http://www.open3d.org/docs/latest/python_api/open3d.pipelines.registration.registration_ransac_based_on_feature_matching.html) from open3d


Try to find the transformation from r1 to r2.
Attempt with point to point and point to plane
```Python
point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()
```

When using ransac focus on the arguments below the rest are optional
```Python
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point)
```

In [5]:
####
# Call RANSAC here
####

# ransac_result = ...
# draw_registrations(source, target, ransac_result.transformation, True)

distance_threshold = 10
point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(with_scaling=False)
ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample,
    source_fpfh, target_fpfh, 1, 
    distance_threshold,
    point_to_point)
draw_registrations(source, target, ransac_result.transformation, True)

## Exercises
### A)
    Can you get a decent transformation from r1 to r3?
### B)
    with the following checkers can you get better results from RANSAC? Try tweaking the parameters of them Can you make Point to Plane work do not spend too long on this if you cant skip it. (I was not able to get a good fit)

You can also try tweaking the voxel_size
```Python
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

cecker_list = [c0,c1,c2]

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point,
    checkers = checker_list)
```


In [6]:
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0,c1,c2]

ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point,
    checkers = checker_list)

draw_registrations(source, target, ransac_result.transformation, True)